In [ ]:
import metmhn.regularized_optimization as reg_opt
import metmhn.Utilityfunctions as utils

import pandas as pd
import warnings
warnings.simplefilter(action='ignore', 
                      category=pd.errors.PerformanceWarning)

import jax.numpy as jnp
import numpy as np
import jax as jax
jax.config.update("jax_enable_x64", True)

import logging
# Adapt path to where logs should be kept
logging.basicConfig(filename='../logs/paad.log',
                    format='%(asctime)s %(levelname)-8s %(message)s',
                    filemode='w', 
                    level=logging.INFO, 
                    force=True,
                    datefmt='%Y-%m-%d %H:%M:%S'
                    )

In the following example, we have information about the genotypes of Primary Tumors (PTs) and Metastases (MTs) for patients suffering from Pancreatic adenocarcinomas (PAADs) and Pancreatic neuroendocrine tumors (PANETs). For some patients we only know the status of either the PT or the MT. This is indicated in our data in the column 'isPaired'. The column 'metaStatus' indicates the type of the tumor ('present', 'isMetastasis', 'absent') if only a single genotype is available.

In [ ]:
mut_handle = "../data/luad/G13_LUAD_PM_v2_Events_19and11_Full.csv"
annot_handle = "../data/luad/G13_LUAD_PM_v2_sampleSelection_19and11.csv"
annot_data = pd.read_csv(annot_handle)
mut_data = pd.read_csv(mut_handle)
mut_data.rename(columns={"Unnamed: 0":"patientID"}, inplace = True)
dat = pd.merge(mut_data, annot_data.loc[:, ['patientID', 'metaStatus']], 
               on=["patientID", "patientID"])
#muts = ['P.KRAS (M)', 'M.KRAS (M)', 'P.TP53 (M)', 'M.TP53 (M)',
#       'P.SMAD4/18q (Del)', 'M.SMAD4/18q (Del)', 'P.MYC/8q (Amp)', 'M.MYC/8q (Amp)',
#       'P.TP53/17p (Del)', 'M.TP53/17p (Del)', 'P.SMAD4 (M)', 'M.SMAD4 (M)', 
#       'P.KRAS/12p (Amp)', 'M.KRAS/12p (Amp)', 'P.SETD2/3p (Del)', 'M.SETD2/3p (Del)',
#       'P.MEN1/11q (Del)', 'M.MEN1/11q (Del)',
#       'P.RNF43 (M)', 'M.RNF43 (M)', 'P.MEN1 (M)', 'M.MEN1 (M)', 
#       'P.GNAS (M)', 'M.GNAS (M)', 'P.DAXX (M)', 'M.DAXX (M)', 'P.KMT2C (M)',
#       'M.KMT2C (M)', 'P.ATM (M)', 'M.ATM (M)', 'P.RBM10 (M)', 'M.RBM10 (M)',
#       'P.BRCA2 (M)', 'M.BRCA2 (M)', 'P.ATRX (M)', 'M.ATRX (M)',
#       'P.PIK3CA (M)', 'M.PIK3CA (M)', 'P.SETD2 (M)', 'M.SETD2 (M)',
#       'P.TGFBR1 (M)', 'M.TGFBR1 (M)', 'P.RB1 (M)', 'M.RB1 (M)',
#       'P.SMARCA4 (M)', 'M.SMARCA4 (M)', 'P.SMAD3 (M)', 'M.SMAD3 (M)',
#       'P.MAP2K4 (M)', 'M.MAP2K4 (M)', 'P.TSC2 (M)', 'M.TSC2 (M)',
#       'P.BCOR (M)', 'M.BCOR (M)', 'P.STK11 (M)', 'M.STK11 (M)', 'P.U2AF1 (M)',
#       'M.U2AF1 (M)', 'P.SF3B1 (M)', 'M.SF3B1 (M)', 'P.PTPRT (M)',
#       'M.PTPRT (M)']
muts = list(dat.columns[1:-4])
mut_data.columns

In [ ]:
# Label each datapoint with a numeric value according to its sequencetype
dat["type"] = dat.apply(utils.categorize, axis=1)
dat["Seeding"] = dat.apply(utils.add_seeding, axis=1)
dat.loc[dat["M.AgeAtSeqRep"] == "No metastasis included", "M.AgeAtSeqRep"] = pd.NA
dat.loc[dat["P.AgeAtSeqRep"] == "No primary included", "P.AgeAtSeqRep"] = pd.NA
dat["P.AgeAtSeqRep"] = dat["P.AgeAtSeqRep"].astype(pd.Int64Dtype())
dat["M.AgeAtSeqRep"] = dat["M.AgeAtSeqRep"].astype(pd.Int64Dtype())
dat["diag_order"] = dat["M.AgeAtSeqRep"] - dat["P.AgeAtSeqRep"]
dat["diag_order"] = dat["diag_order"].apply(lambda x: pd.NA if pd.isna(x) else 2 if x < 0 else 1 if x > 0 else 0) 
dat["diag_order"] = dat["diag_order"].astype(pd.Int64Dtype())
events_data = muts+["Seeding"]
# Only use datapoints where the state of the metastasis is known
cleaned = dat.loc[dat["type"].isin([0,1,2,3]), muts+["Seeding", "type", "diag_order"]]
# Remove completely empty datapoints
cleaned.drop(cleaned[cleaned.iloc[:,:-2].sum(axis=1)<1].index, inplace=True)
# Remove coupled datapoints where the diagnosis_order is unknown 
cleaned.drop(cleaned[(cleaned["type"] == 3) & (pd.isna(cleaned["diag_order"]) == True)].index, inplace=True)
dat_prim_nomet, dat_prim_met, dat_met_only, dat_coupled = utils.split_data(cleaned, events_data+["diag_order"])
dat.columns

Retrieve the event names and trim the PT/MT identifier:

In [ ]:
events_plot = []
for elem in cleaned.columns[:-3].to_list()[::2]:
    full_mut_id = elem.split(".")
    events_plot.append(full_mut_id[1])
events_plot.append("Seeding")

Enumerate the frequencies of SNVs and CNVs in all subgroups. 'NM/EM' refer to Never Metastasizing/ Ever Metastasizing tumors, where only a single genotype is known. A Mutation is referred to as 'MT/PT-private' if happens exclusively in the MT or PT, otherwise it is called 'shared':

In [ ]:
n_tot = (cleaned.shape[1]-1)//2 + 1
n_mut = n_tot-1
utils.marg_frequs(dat_prim_nomet[:,:-1], dat_prim_met[:,:-1], dat_met_only[:,:-1], dat_coupled[:,:-1], events_plot)

Optional: We use a  sparsity promoting L1 penalty. The weight of the penalization can be determined in a k-fold crossvalidation:

In [ ]:
#log_lams = np.linspace(-4, -2, 5)
#lams = 10**log_lams
#print(lams)
#utils.cross_val(cleaned.copy(), events_data, lams, 5, 0.83)

In [ ]:
import  matplotlib.pyplot as plt
iwas = dat_paired.dropna()
np.bincount(iwas["diag_order"])

In [ ]:
dat_paired.iloc[0:10,:]

Train an MHN on the full dataset:

In [ ]:
dat_coupled = jnp.tile(jnp.nan, 68).reshape((1,-1))
#dat_prim_met = jnp.tile(jnp.nan, 56).reshape((1,-1))
#dat_met_only = jnp.tile(jnp.nan, 56).reshape((1,-1))

penal = 0.002 #L1 penalty on off-diagonals
m_p_corr = 0.48
th_init, dp_init, dm_init = utils.indep(jnp.array(cleaned[events_data].to_numpy()), dat_coupled.shape[0])
theta, d_p, d_m= reg_opt.learn_mhn(th_init, dp_init, dm_init, dat_prim_nomet, dat_prim_met, dat_met_only,
                                    dat_coupled, m_p_corr, penal)

In [ ]:
th_plot = np.row_stack((d_p.reshape((1,-1)), 
                        d_m.reshape((1,-1)), 
                        theta))

Visualize the results:

In [ ]:
utils.plot_theta(th_plot, events_plot, .1)

In [ ]:
df2 = pd.DataFrame(th_plot, columns=events_plot)
df2.to_csv("../results/prad/sim.csv")
